# Modeling

This notebook looks at different implementations of Learning-to-rank (LTR) algorithms to see if we can obtain better recommendations than through the other basic techniques laid out in the 'eda.ipynb' notebook.

* The first task will be to add a few features to the dataset (mainly the 'relevancy' column) for LambdaRank to work properly.

* The second section will use PyTorch to implement the RankNet algorithm.

* The third section will implement LambdaRank via XGBoost and LightGBM.

* The fourth section will use pre-trained LLM's to see if rankings can be gotten, via prompt engineering.

In [104]:
import h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import random
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings(action='ignore')

## reads in the sentence embeddings in
with h5py.File('../Save_Models/bert_embeddings.h5', 'r') as f:
    sentence_embeddings = f['bert_embeddings'][:]

print(f"Length of the overall dataset: {len(sentence_embeddings)}\n")
print(f"Length of each vector in each row: {len(sentence_embeddings[0])}\n")
print(f"First 5 vectors in the dataset:")
print(sentence_embeddings[:5])

Length of the overall dataset: 104

Length of each vector in each row: 768

First 5 vectors in the dataset:
[[ 0.04244611  0.20658404  0.3496604  ...  0.08637957  0.16035193
   0.11486406]
 [-0.17883101 -0.19114071  0.11720981 ...  0.0303745  -0.00377608
   0.12247177]
 [-0.12000985 -0.45496473  0.48939645 ...  0.17277648  0.07609212
  -0.3418459 ]
 [-0.40924832 -0.49934703  0.01931042 ...  0.0454201   0.19822782
  -0.43201998]
 [-0.19579899 -0.16278872 -0.30866104 ...  0.1933579   0.28864014
  -0.1512861 ]]


In [105]:
## reads in the altered dataset with job_titles stripped of stopwords and other unecessary words
df = pd.read_csv('../Data/alterned_job_talents.csv')
df.drop(labels=['Unnamed: 0', 'fit'], axis=1, inplace=True)
df['vector_embeddings'] = sentence_embeddings.tolist()
df.head()

,id,job_title,connections,location,vector_embeddings
0,1,ct bauer college business graduate magna cum l...,85,"Houston, Texas","[0.042446110397577286, 0.20658403635025024, 0...."
1,2,native english teacher epik english program korea,500,Kanada,"[-0.17883101105690002, -0.19114071130752563, 0..."
2,3,aspiring human resources professional,44,"Raleigh-Durham, North Carolina Area","[-0.12000985443592072, -0.4549647271633148, 0...."
3,4,people development coordinator ryan,500,"Denton, Texas","[-0.40924832224845886, -0.4993470311164856, 0...."
4,5,advisory board member celal bayar university,500,"İzmir, Türkiye","[-0.19579899311065674, -0.16278871893882751, -..."


In [106]:
## reads in the original dataset and stores the original job_titles into a list for later on
og_df = pd.read_csv('../Data/potential-talents - Aspiring human resources - seeking human resources.csv')
og_df.drop(labels='fit', axis=1, inplace=True)
og_df_job_title = og_df['job_title'].tolist()
og_df_id = og_df['id']

In [107]:
SEED = random.randint(1000,9999)
SEED = 6992

In [108]:
##### this step makes a dataframe that compares each candidates' cosine similarity score to every other candidate
sentence_similarities = cosine_similarity(sentence_embeddings, sentence_embeddings)
sentence_similarities = sentence_similarities.round(6)
sim_df = pd.DataFrame(sentence_similarities, columns=df['job_title'], index=df['job_title'])
sim_df.head(2)

job_title,ct bauer college business graduate magna cum laude aspiring human resources professional,native english teacher epik english program korea,aspiring human resources professional,people development coordinator ryan,advisory board member celal bayar university,aspiring human resources specialist,student humber college aspiring human resources generalist,hr senior specialist,student humber college aspiring human resources generalist,seeking human resources hris generalist positions,...,student westfield state university,student indiana university kokomo business management retail manager delphi hardware paint,aspiring human resources professional,student,seeking human resources position,aspiring human resources manager graduating seeking entry level human resources position st louis,human resources generalist loparex,business intelligence analytics travelers,set success,director administration excellence logging
job_title,,,,,,,,,,,,,,,,,,,,,
ct bauer college business graduate magna cum laude aspiring human resources professional,1.000000,0.779731,0.815985,0.655214,0.676571,0.838286,0.862277,0.768637,0.862277,0.701005,...,0.769431,0.654929,0.815985,0.751647,0.657489,0.756026,0.754981,0.682939,0.638996,0.712136
native english teacher epik english program korea,0.779731,1.000000,0.755116,0.737595,0.746116,0.770067,0.777830,0.749807,0.777830,0.741068,...,0.757100,0.720969,0.755116,0.792102,0.729569,0.726489,0.736004,0.700008,0.676520,0.738598


In [109]:
## takes the average of all similarity scores of a job_title between all other job_titles 
mean_sim_scores = []
for row in list(sim_df.iterrows()):
    row = row[1]
    mean_sim_score = np.mean(row.values)
    mean_sim_scores.append(mean_sim_score)

In [110]:
## creates a new column to the dataframe for mean_sim_scores list
df['average_cosine_score'] = mean_sim_scores
df.head()

,id,job_title,connections,location,vector_embeddings,average_cosine_score
0,1,ct bauer college business graduate magna cum l...,85,"Houston, Texas","[0.042446110397577286, 0.20658403635025024, 0....",0.729857
1,2,native english teacher epik english program korea,500,Kanada,"[-0.17883101105690002, -0.19114071130752563, 0...",0.723307
2,3,aspiring human resources professional,44,"Raleigh-Durham, North Carolina Area","[-0.12000985443592072, -0.4549647271633148, 0....",0.728617
3,4,people development coordinator ryan,500,"Denton, Texas","[-0.40924832224845886, -0.4993470311164856, 0....",0.689792
4,5,advisory board member celal bayar university,500,"İzmir, Türkiye","[-0.19579899311065674, -0.16278871893882751, -...",0.660760


In [111]:
## creates a dataframe that houses each job titles' vector embedding for training and testing purposes
embeddings_df = pd.DataFrame(sentence_embeddings.tolist())
embeddings_df['cosine_score'] = df['average_cosine_score']
embeddings_df['connection'] = df['connections']
embeddings_df['job_title'] = df['job_title']
embeddings_df['id'] = df['id']
embeddings_df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,762,763,764,765,766,767,cosine_score,connection,job_title,id
0,0.042446,0.206584,0.349660,-0.068304,0.532296,0.061598,-0.073295,0.059054,0.056990,-0.044459,...,0.266532,-0.202613,-0.104868,0.086380,0.160352,0.114864,0.729857,85,ct bauer college business graduate magna cum l...,1
1,-0.178831,-0.191141,0.117210,-0.061379,-0.101844,-0.005244,0.047576,0.280545,0.067215,-0.260616,...,-0.065128,0.030020,0.164532,0.030375,-0.003776,0.122472,0.723307,500,native english teacher epik english program korea,2
2,-0.120010,-0.454965,0.489396,0.091429,0.735099,-0.011250,0.020545,0.385089,0.333931,-0.132409,...,0.157942,-0.197309,-0.162478,0.172776,0.076092,-0.341846,0.728617,44,aspiring human resources professional,3


In [112]:
'''
this step is for later on when we go to evaluate the LGMBRanker and XGBRanker models

'''

## looking at job titles that have human resources in the title 
hr_df = embeddings_df[embeddings_df['job_title'].str.contains('human resources')]
## creates a binary feature that corresponds to whether the title has 'aspiring human resources' (1) and not (0)
hr_df['relevancy'] = hr_df['job_title'].str.contains('aspiring human resources').astype(int)
## replaces the 0-1 values with 1-2
hr_df.replace({'relevancy': {0:1, 1:2}}, inplace=True)

print(f"Length of the 'human resources' dataframe: {len(hr_df)}")
hr_df_id = hr_df['id'] ## ids of hr_df which are used to filter those candidates from the others that do not have 'aspiring human resources' in their title

## creates another dataframe that are the candidates whose job titles do not have 'aspiring human resources' in their job_title
non_relevant_cands = embeddings_df.drop(labels=hr_df_id-1, axis=0)
non_relevant_cands['relevancy'] = non_relevant_cands['job_title'].str.contains('1').astype(int) ## I just needed to create the relevancy column for the next step (forget '1')
print(f"Length of the NON-'human resources' dataframe: {len(non_relevant_cands)}")

## adds the two dataframes back together 
df = pd.concat([hr_df, non_relevant_cands]) 
df.tail()

Length of the 'human resources' dataframe: 61
Length of the NON-'human resources' dataframe: 43


,0,1,2,3,4,5,6,7,8,9,...,763,764,765,766,767,cosine_score,connection,job_title,id,relevancy
95,0.527253,-0.556477,-0.204190,0.105200,0.214329,0.365075,-0.518916,0.413426,0.529036,-0.281506,...,-0.306618,-0.051233,-0.060531,-0.315960,-0.200867,0.655882,19,student indiana university kokomo business man...,96,0
97,0.040964,-0.090720,0.277195,0.187189,0.338038,-0.096053,-0.169334,0.142951,0.279250,-0.409109,...,-0.177394,-0.468369,0.078479,-0.091068,0.239894,0.696758,4,student,98,0
101,-0.060670,-0.463036,0.098590,0.232495,0.271226,0.144223,-0.360194,0.348765,0.138774,-0.358163,...,-0.257512,-0.086818,-0.221545,-0.132243,0.062259,0.660679,49,business intelligence analytics travelers,102,0
102,-0.043576,-0.429659,0.429790,0.280550,-0.321894,0.271992,0.332883,-0.096879,-0.055402,-0.079595,...,0.079564,0.056456,-0.159439,-0.217776,0.022860,0.603045,500,set success,103,0
103,-0.687541,-0.397018,0.083177,0.188525,0.078910,-0.223303,-0.268258,0.384961,0.323239,-0.141671,...,-0.056168,-0.098288,0.141740,-0.007365,-0.029101,0.687625,500,director administration excellence logging,104,0


In [113]:
## saves the pre-ranked dataframe
df.to_csv('../Data/pre-ranked_df.csv')

To make this a little more clear, I needed to pre-rank the candidates on a certain phrase or job title description (this phrase can be changed depending on the job title). So what the first part does is look at the more 'generic' part of the description - in this case 'human resources' - because if it was too specific e.g. 'aspiring human resources', then only a certain few candidates would be selected.

With that generic description we can partition out the more specific candidates. And in this case give them a ranking of 2 - relevant and 1 - sort or relevant.

The candidates who do not have that generic title in their job description will be ranked 0 - not relevant. And the two dataframes will be concatenated to one another to make a pre-ranked dataframe for later on when we go to use the LGBM and XGB models for LambdaRank.

In [114]:
relevancy_counts = df['relevancy'].value_counts()

print(f"There are {relevancy_counts[1]} candidates that have 'aspiring human resources' in their job title.\nThere are {relevancy_counts[2]} candidates that have 'human resources' in their job title.\nAnd there are {relevancy_counts[0]} candidates that have neither of those phrases in their job title.")

There are 26 candidates that have 'aspiring human resources' in their job title.
There are 35 candidates that have 'human resources' in their job title.
And there are 43 candidates that have neither of those phrases in their job title.


In [115]:
## this dataframe will be used for the final ranking of candidates after the model has been trained
''' the process here is to pre-rank the candidates based upon our preliminary ranking
 of the phrase 'aspiring human resources' as well as the cosine similarity scores'''

sorted_final_df_ = df.sort_values(by=['relevancy', 'cosine_score', 'connection'], ascending=False) ## sorts by relevancy (2 is highest, then 1, then 0's) and then sorts of cosine_score featue
sorted_final_df_label = sorted_final_df_['relevancy'].astype('float') ## changes the dtype to float
sorted_final_df_id = sorted_final_df_['id'] ## Series that will be used for indexing by the id later in the LambdaRanking section
sorted_final_df = sorted_final_df_.drop(labels=['relevancy', 'id', 'job_title'], axis=1) ## drops the unecessary columns for training and prediction

print(f"Shape of final dataset: {sorted_final_df.shape}")

sorted_final_df.head()

Shape of final dataset: (104, 770)


,0,1,2,3,4,5,6,7,8,9,...,760,761,762,763,764,765,766,767,cosine_score,connection
5,-0.023534,-0.331757,0.676681,0.202042,0.628635,0.056037,-0.053359,0.20367,0.365207,-0.09451,...,-0.090004,0.048109,0.149887,-0.130065,-0.105329,0.202839,0.01202,-0.260334,0.737194,1
23,-0.023534,-0.331757,0.676681,0.202042,0.628635,0.056037,-0.053359,0.20367,0.365207,-0.09451,...,-0.090004,0.048109,0.149887,-0.130065,-0.105329,0.202839,0.01202,-0.260334,0.737194,1
35,-0.023534,-0.331757,0.676681,0.202042,0.628635,0.056037,-0.053359,0.20367,0.365207,-0.09451,...,-0.090004,0.048109,0.149887,-0.130065,-0.105329,0.202839,0.01202,-0.260334,0.737194,1
48,-0.023534,-0.331757,0.676681,0.202042,0.628635,0.056037,-0.053359,0.20367,0.365207,-0.09451,...,-0.090004,0.048109,0.149887,-0.130065,-0.105329,0.202839,0.01202,-0.260334,0.737194,1
59,-0.023534,-0.331757,0.676681,0.202042,0.628635,0.056037,-0.053359,0.20367,0.365207,-0.09451,...,-0.090004,0.048109,0.149887,-0.130065,-0.105329,0.202839,0.01202,-0.260334,0.737194,1


The step above takes the dataset and sorts the dataset, additionally, on their respective cosine similarity scores and their 'connection' number.

The dataset 'df' in a couple of the cell's above is already sorted by relevancy - 2,1,0, I thought the additional sorting might be helpful.

In [116]:
## permutates the embeddings dataframe for testing and training purposes
permutation_df = np.random.permutation(df)
permutation_df = pd.DataFrame(permutation_df, columns=df.columns)
permutation_df.drop(labels=['job_title'], axis=1, inplace=True)
permutation_df = permutation_df.astype('float')

permutation_df.head()

,0,1,2,3,4,5,6,7,8,9,...,762,763,764,765,766,767,cosine_score,connection,id,relevancy
0,0.062218,-0.286468,-0.354052,-0.215897,0.445618,0.258911,-0.081893,0.319557,0.227221,-0.278935,...,0.229561,-0.017427,-0.243956,0.105172,0.040582,0.107361,0.684660,2.0,11.0,0.0
1,-0.023534,-0.331757,0.676681,0.202042,0.628635,0.056037,-0.053359,0.203670,0.365207,-0.094510,...,0.149887,-0.130065,-0.105329,0.202839,0.012020,-0.260334,0.737194,1.0,49.0,2.0
2,0.345657,0.016201,0.479107,0.011839,0.502112,0.158452,0.048503,0.190604,0.182019,-0.118587,...,0.233318,-0.293753,-0.041762,-0.003324,0.008638,-0.086306,0.720686,61.0,37.0,2.0
3,-0.115044,-0.344139,-0.160638,0.030973,-0.122298,0.337532,-0.150123,0.440144,0.444860,-0.162885,...,0.273708,-0.347141,0.022418,0.146357,0.268260,-0.148825,0.673780,455.0,81.0,1.0
4,0.402807,0.183804,0.213086,0.151948,0.444634,0.040844,-0.118473,-0.163188,-0.009283,-0.154639,...,0.246188,-0.290659,-0.366607,-0.272215,-0.200631,0.068429,0.699317,82.0,70.0,1.0


In [117]:
train_number = int((len(permutation_df)*.80)-3)

## these training and testing sets are comprised of the train_label - the column that shows whether the job title has the words 'aspiring human resources' in the title; train_id - the column that has each candidates' unique id; and train - the overall features that will be used for training the neural network

## the ids will be used to see which candidates got chosen for comparison during training

train = permutation_df[:train_number]
train_label = train['relevancy'].to_numpy()
train_id = train['id'].astype('int32').to_numpy()
train = train.drop(labels=['relevancy', 'id'], axis=1).to_numpy()

test = permutation_df[train_number:]
test_label = test['relevancy'].to_numpy()
test_id = test['id'].astype('int32').to_numpy()
test = test.drop(labels=['relevancy', 'id'], axis=1).to_numpy() ## converts all pd.DataFrames/pd.Series into numpy.ndarray for next cell

print(f"Train shape: {train.shape} | Train label shape: {train_label.shape} | Test shape: {test.shape} | Test label shape: {test_label.shape}")

Train shape: (80, 770) | Train label shape: (80,) | Test shape: (24, 770) | Test label shape: (24,)


In [118]:
from torch.utils.data import DataLoader, TensorDataset

# train_ and test_dataloader will be used for training and evaluating the neural network

train_dataset = TensorDataset(torch.tensor(train), torch.tensor(train_label), torch.tensor(train_id))
test_dataset = TensorDataset(torch.tensor(test), torch.tensor(test_label), torch.tensor(test_id))
final_dataset = TensorDataset(torch.tensor(sorted_final_df.to_numpy()), torch.tensor(sorted_final_df_id.to_numpy()))

train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=False)
final_dataloader = DataLoader(final_dataset, batch_size=2, shuffle=True)

## Pytorch

This link was a good guide for implementing RankNet via PyTorch:

https://medium.com/@mandeep0405/learning-to-rank-ranknet-simplified-5d7f7334133d

In [119]:
from torch import nn
from torch import functional as F
import torch.optim as optim
import torchmetrics
from torchmetrics import Accuracy, Recall, F1Score
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import ndcg_score
from sklearn.preprocessing import MultiLabelBinarizer
# torch.autograd.detect_anomaly(True)

####### RankNet implementation utilizing PyTorch ########
class RankNet(nn.Module):
    def __init__(self, num_feature):
        super(RankNet, self).__init__()
        self.input = nn.Linear(num_feature, 256)
        self.hidden_1 = nn.Linear(256, 256)
        self.hidden_2 = nn.Linear(256, 256)
        self.output = nn.Linear(256, 1)

        self.dropout = nn.Dropout(p=0.3)
        self.norm_layer = nn.LayerNorm(512)
        self.activation = nn.ReLU()

        self.sigmoid = nn.Sigmoid()

    def forward(self, x1, x2):
        # Process first item
        in_x1 = self.input(x1)
        h1_x1 = self.activation(self.dropout(self.hidden_1(in_x1)))
        h2_x1 = self.activation(self.dropout(self.hidden_2(h1_x1)))
        out1 = self.output(h2_x1)

        # Process second item
        in_x2 = self.input(x2)
        h1_x2 = self.activation(self.dropout(self.hidden_1(in_x2)))
        h2_x2 = self.activation(self.dropout(self.hidden_2(h1_x2)))
        out2 = self.output(h2_x2)

        # returns both candidate scores
        return (out1, out2)

    def cross_entropy_loss(self, out1, out2):
        # Decide whether x1 (out1) should be ranked higher than x2 (out2)
        if out1 > out2:
            p_hat_ij = 1.0
        elif out1 < out2:
            p_hat_ij = 0.0
        else:
            p_hat_ij = 0.5

        # Obtain probability whether x1 is greater than x2
        p_ij = self.sigmoid(out1 - out2)

        # Cross-Entropy Loss function
        bce_loss = -p_hat_ij * torch.log(p_ij) - (1.0 - p_hat_ij) * torch.log(1.0 - p_ij)
        # mean_bce_loss = np.mean(bce_loss)

        return (p_ij, bce_loss)

In [120]:
def train_pt(model, optimizer, train_data, test_data, epochs=100):
    '''
    -- Parameters --
    model: PyTorch RankNet model
    optimizer: PyTorch optimizer
    train_data: PyTorch Dataloader object used for training
    test_data: Pytorch Dataloader object used for evaluation
    epochs: set to 250 but can be changed if need be

    -- Returns --
    prints the epoch and loss for every pass through the network as well as a graph that charts the training loss/epoch
    '''
    model.train()
    training_loss = []
    correct_predictions = 0
    accuracy_scores = []

    for epoch in range(epochs):
        train_loss = 0.0
        for batch in train_data:
            data, _, _ = batch
            ## converts the data tensor to list for indexing purposes
            data = data.tolist()
            
            optimizer.zero_grad()
            
            data1_out, data2_out = model.forward(torch.tensor(data[0]).reshape(1,-1), torch.tensor(data[1]).reshape(1,-1))

            p_ij, loss = model.cross_entropy_loss(data1_out, data2_out)

            if p_ij > .50:
                correct_predictions += 1
            train_accuracy = correct_predictions / len(train_data)

            loss.backward(gradient=[torch.tensor(1e-6, dtype=torch.float).reshape(1, -1)])
            optimizer.step()

            train_loss += loss.item()

            ## evaluation
            test_loss = []
            val_accuracy_scores = []
            val_correct_predictions = 0
            val_loss = 0.0
            model.eval()

            with torch.no_grad():
                for batch in test_data:
                    data, _, _  = batch
                    data = data.tolist()
                    
                    data1_out, data2_out = model.forward(torch.tensor(data[0]).reshape(1,-1), torch.tensor(data[1]).reshape(1,-1))

                    p_ij, loss = model.cross_entropy_loss(data1_out, data2_out)

                    val_loss += loss.item()

                    if p_ij > .50:
                        val_correct_predictions += 1
                    val_accuracy = val_correct_predictions / len(test_data)

                eval_loss = val_loss / len(test_data)
                test_loss.append(eval_loss)
                val_accuracy_scores.append(val_accuracy)
                model.train()

        train_loss = train_loss / len(train_data)
        training_loss.append(train_loss)
        accuracy_scores.append(train_accuracy)

        print(f"Epoch {epoch}\nTraining Loss: {train_loss} | Evaluation Loss: {eval_loss}")
        print() 

    ## plots the training loss over all epochs
    # plt.plot(val_accuracy_scores, label='Validation Accuracy')
    # plt.plot(accuracy_scores, label='Training Accuracy')
    # plt.title('Training Loss by Epoch')
    # plt.legend()
    # plt.xlabel('Epoch')
    # plt.ylabel('Training Loss')
    # plt.show()


In [121]:
def final_ranking(model, dataloader, epochs=100):
    '''
    -- Parameters --
    model: PyTorch model
    dataloader: PyTorch Dataloader object
    epochs: set to 100 but can be changed if need be
    
    -- Returns --
    candidate id's from the final batch. 
     - Theoretically, the model has compared each candidate to each other candidate and this last batch is the final ranking from the model
    '''
    id_1 = []
    id_2 = []
    score = []
    ids = []
    model.eval()

    with torch.no_grad():
        for epoch in range(epochs):
            for batch in dataloader:
                data, id  = batch
                data = data.tolist()
                id = id.tolist()
                
                # id_1.append(id[0])
                # id_2.append(id[1])
                
                data1_out, data2_out = model.forward(torch.tensor(data[0]).reshape(1,-1), torch.tensor(data[1]).reshape(1,-1))

                p_ij, loss = model.cross_entropy_loss(data1_out, data2_out)

                if epoch == epochs-1: ## final batch results
                    if p_ij > .50:
                        ids.append(id[0])
                        ids.append(id[1])
                    else:
                        ids.append(id[1])
                        ids.append(id[0])

    return np.array(ids)

    '''these four lines would be to print out the dataframe with the ids of the two candidates and the probability score that the model gave
    #### make sure to uncomment the id_1.append and id_2.append lines
    '''
    #             y = 1 if p_ij >= .50 else 0
    #             score.append(y)
    
    # df = pd.DataFrame({'id_1': id_1, 'id_2': id_2, 'probability_score': score})
    # return df.head(len(dataloader))


In [122]:
n_feature = train.shape[1]

rank_model = RankNet(num_feature=n_feature)
optimizer = optim.SGD(lr=.001, params=rank_model.parameters(), weight_decay=1e-4)

In [123]:
train_pt(model=rank_model, optimizer=optimizer, train_data=train_dataloader, test_data=test_dataloader)

Epoch 0
Training Loss: 0.3625580064486712 | Evaluation Loss: 0.40594834213455516

Epoch 1
Training Loss: 0.3722001419402659 | Evaluation Loss: 0.4059494572381179

Epoch 2
Training Loss: 0.32036415170878174 | Evaluation Loss: 0.4059506691992283

Epoch 3
Training Loss: 0.359970722370781 | Evaluation Loss: 0.40595169365406036

Epoch 4
Training Loss: 0.3451033216901124 | Evaluation Loss: 0.40595339114467305

Epoch 5
Training Loss: 0.37094847375992684 | Evaluation Loss: 0.40595469747980434

Epoch 6
Training Loss: 0.37505601667799054 | Evaluation Loss: 0.40595657130082446

Epoch 7
Training Loss: 0.3511357634328306 | Evaluation Loss: 0.4059583283960819

Epoch 8
Training Loss: 0.3653770822100341 | Evaluation Loss: 0.40595967322587967

Epoch 9
Training Loss: 0.37383457282558086 | Evaluation Loss: 0.4059610354403655

Epoch 10
Training Loss: 0.40344672673381865 | Evaluation Loss: 0.4059629167119662

Epoch 11
Training Loss: 0.32101190818939357 | Evaluation Loss: 0.4059644987185796

Epoch 12
Traini

In [124]:
## saves the rank_net model
torch.save(rank_model, f='../Save_Models/rank_net.pt')

In [125]:
'''
- final rankings (by id) from the model
'''
rank_net_model = torch.load(f='../Save_Models/rank_net.pt')
final_eval = final_ranking(model=rank_net_model, dataloader=final_dataloader)
final_eval

array([ 80,  82,  30,  71,  95,  50,  99,  42,  54,  33,  88,  85,  15,
         2, 102,  43,  41,  73,  98,  68,  24,  57,   7,  67,   5,  38,
        70,  78,  74,  55,  81, 104,   3, 101,  91,  18,  44,  32,  62,
        61,  28,  94,  17,  40,   1,  69,  92,  83,  60,  89,  86,  29,
        11, 100,  96,  65,  56,  12,  87,  77,  66,  51,  46,  47,  58,
        39,  14,  10,   9,  16,  13,  35,  84,  48,  19,  27,  36,  25,
         6,  37,  93,  64,  23,  20,  52,  75,   8,  22,  49,  21,  53,
         4,  97,  31,  79, 103,  63,  72,  76,  26,  34,  45,  90,  59])

In [126]:
og_df.loc[final_eval-1].head(40)

,id,job_title,location,connection
79,80,Junior MES Engineer| Information Systems,"Myrtle Beach, South Carolina Area",52
81,82,Aspiring Human Resources Professional | An ene...,"Austin, Texas Area",174
29,30,Seeking Human Resources Opportunities,"Chicago, Illinois",390
70,71,"Human Resources Generalist at ScottMadden, Inc.","Raleigh-Durham, North Carolina Area",500+
94,95,Student at Westfield State University,"Bridgewater, Massachusetts",57
49,50,Student at Humber College and Aspiring Human R...,Kanada,61
98,99,Seeking Human Resources Position,"Las Vegas, Nevada Area",48
41,42,"SVP, CHRO, Marketing & Communications, CSR Off...","Houston, Texas Area",500+
53,54,Student at Chapman University,"Lake Forest, California",2
32,33,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44


In [127]:
## use this cell and next cell ONLY if you uncomment the lines in the final_ranking function

# test_candidates_1 = final_eval['id_1'].unique()
# df.iloc[test_candidates_1 - 1].tail(10)

In [128]:
# test_candidates_2 = final_eval['id_2'].unique()
# df.iloc[test_candidates_2 - 1].tail(10)

## LambdaRank

Using this link as a guide - https://forecastegy.com/posts/lightgbm-learning-to-rank-python/, I was able to implement a LambdaRank model using both XGBoost and LightGBM.

In [129]:
import lightgbm as lgb 
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import ndcg_score

SEED = 7247

train = permutation_df[:train_number]
train_group = train.groupby(by=['relevancy'])['relevancy'].count().to_numpy() ## counts the relevancy scores (0 and 1) 
X_train = train.drop(labels=['relevancy', 'id'], axis=1) ## drops the two features that are not relevant for training
y_train = train['relevancy'].to_numpy() ## target vector
print(f"Train group sizes (first corresponds to the candidates whose relevancy score is 2, 1, 0): {train_group}")

test = permutation_df[train_number:]
test_group = test.groupby(by=['relevancy'])['relevancy'].count().to_numpy()
X_test = test.drop(labels=['relevancy', 'id'], axis=1)
y_test = test['relevancy'].to_numpy()
print(f"Test group sizes: {test_group}\n")

print(f"X_train shape: {X_train.shape}\ny_train shape: {y_train.shape}\nX_test shape: {X_test.shape}\ny_test shape: {y_test.shape}")

X_train.head()

Train group sizes (first corresponds to the candidates whose relevancy score is 2, 1, 0): [34 19 27]
Test group sizes: [9 7 8]

X_train shape: (80, 770)
y_train shape: (80,)
X_test shape: (24, 770)
y_test shape: (24,)


,0,1,2,3,4,5,6,7,8,9,...,760,761,762,763,764,765,766,767,cosine_score,connection
0,0.062218,-0.286468,-0.354052,-0.215897,0.445618,0.258911,-0.081893,0.319557,0.227221,-0.278935,...,0.326511,0.056631,0.229561,-0.017427,-0.243956,0.105172,0.040582,0.107361,0.684660,2.0
1,-0.023534,-0.331757,0.676681,0.202042,0.628635,0.056037,-0.053359,0.203670,0.365207,-0.094510,...,-0.090004,0.048109,0.149887,-0.130065,-0.105329,0.202839,0.012020,-0.260334,0.737194,1.0
2,0.345657,0.016201,0.479107,0.011839,0.502112,0.158452,0.048503,0.190604,0.182019,-0.118587,...,-0.000100,0.107381,0.233318,-0.293753,-0.041762,-0.003324,0.008638,-0.086306,0.720686,61.0
3,-0.115044,-0.344139,-0.160638,0.030973,-0.122298,0.337532,-0.150123,0.440144,0.444860,-0.162885,...,0.046516,0.000962,0.273708,-0.347141,0.022418,0.146357,0.268260,-0.148825,0.673780,455.0
4,0.402807,0.183804,0.213086,0.151948,0.444634,0.040844,-0.118473,-0.163188,-0.009283,-0.154639,...,0.202890,0.064487,0.246188,-0.290659,-0.366607,-0.272215,-0.200631,0.068429,0.699317,82.0


In [130]:
#### LightGBM model
LGBM_ranker = lgb.LGBMRanker(
    random_state=SEED,
    objective='lambdarank',
    metric='pairwise',
    objective_seed=SEED,
    early_stopping=5,
    force_col_wise=True
    )

LGBM_ranker.fit(
    X_train,
    y_train,
    eval_metric=['ndcg'],
    group=train_group,
    eval_set=[(X_test, y_test)],
    eval_group=[test_group],
    eval_at=[5,10]
    )

[LightGBM] [Warning] early_stopping_round is set=5, early_stopping=5 will be ignored. Current value: early_stopping_round=5
[LightGBM] [Info] Total Bins 16300
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 770
[LightGBM] [Warning] early_stopping_round is set=5, early_stopping=5 will be ignored. Current value: early_stopping_round=5
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 5 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wi

LGBMRanker(early_stopping=5, force_col_wise=True, metric='pairwise',
           objective='lambdarank', objective_seed=7247, random_state=7247)

In [131]:
## for the accuracy score metric in the XGBRanker evaluation
def create_multilabels(predictions=list):
    '''
    -- Parameters --
    predictions: a list of continuous floating point numbers

    -- Returns --
    a list of discrete numbers
    '''
    altered_preds = []
    for preds in predictions:
        if preds < 0:
            y = 0
        elif preds > 0 and preds < .50:
            y = 1
        else:
            y = 2
        altered_preds.append(y)
    return altered_preds

In [132]:
preds = LGBM_ranker.predict(sorted_final_df)

preds = create_multilabels(preds)

# mlb = MultiLabelBinarizer()
# preds_multilabel = mlb.fit_transform([preds])

LGBMRanker_predictions = pd.DataFrame({'Candidate_ID': sorted_final_df_id, 'Predictions': preds})
LGBMRanker_predictions = LGBMRanker_predictions.sort_values(by='Predictions', ascending=False)
# lgbm_ndcg_score = ndcg_score(sorted_final_df_label, preds)
# lgbm_ndcg_score = ndcg_score(sorted_final_df_label, y)

# print(f"NDCG Score: {lgbm_ndcg_score}")

print(f"Accuracy: {acc(sorted_final_df_label, preds)}")

og_df.loc[LGBMRanker_predictions['Candidate_ID']-1].head(40)

Accuracy: 0.7211538461538461


,id,job_title,location,connection
5,6,Aspiring Human Resources Specialist,Greater New York City Area,1
32,33,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44
71,72,Business Management Major and Aspiring Human R...,"Monroe, Louisiana Area",5
75,76,Aspiring Human Resources Professional | Passio...,"New York, New York",212
65,66,Experienced Retail Manager and aspiring Human ...,"Austin, Texas Area",57
23,24,Aspiring Human Resources Specialist,Greater New York City Area,1
28,29,Aspiring Human Resources Management student se...,"Houston, Texas Area",500+
26,27,Aspiring Human Resources Management student se...,"Houston, Texas Area",500+
51,52,Student at Humber College and Aspiring Human R...,Kanada,61
49,50,Student at Humber College and Aspiring Human R...,Kanada,61


In [133]:
lambdarank_trauncation_levels = [2,4,8,16,32]
lgbm_ranker2 = lgb.LGBMRanker(
    random_state=SEED,
    objective='rank_xendcg',
    learning_rate=.01,
    sigmoid=.50,
    metric='rank_xendcg',
    early_stopping=5,
    # label_gain=[0,4],
    objective_seed=SEED,
    force_col_wise=True,
    lambdarank_truncation_level=16
    )

lgbm_ranker2.fit(
    X_train,
    y_train,
    eval_metric=['rank_xendcg'],
    group=train_group,
    eval_set=[(X_test, y_test)],
    eval_group=[test_group],
    eval_at=[5,10]
)

[LightGBM] [Warning] early_stopping_round is set=5, early_stopping=5 will be ignored. Current value: early_stopping_round=5
[LightGBM] [Info] Total Bins 16300
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 770
[LightGBM] [Warning] early_stopping_round is set=5, early_stopping=5 will be ignored. Current value: early_stopping_round=5
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 5 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wi

LGBMRanker(early_stopping=5, force_col_wise=True,
           lambdarank_truncation_level=16, learning_rate=0.01,
           metric='rank_xendcg', objective='rank_xendcg', objective_seed=7247,
           random_state=7247, sigmoid=0.5)

In [134]:
preds2 = lgbm_ranker2.predict(sorted_final_df)

LGBMRanker2_predictions = pd.DataFrame({'Candidate_ID': sorted_final_df_id, 'Predictions': preds2})
LGBMRanker2_predictions = LGBMRanker2_predictions.sort_values(by='Predictions', ascending=False)

preds = create_multilabels(preds2)
LGBMRanker2_accuracy_score = acc(sorted_final_df_label, preds)
print(f"Accuracy score: {LGBMRanker2_accuracy_score}")

og_df.loc[LGBMRanker2_predictions['Candidate_ID']-1].head(40)

Accuracy score: 0.40384615384615385


,id,job_title,location,connection
5,6,Aspiring Human Resources Specialist,Greater New York City Area,1
28,29,Aspiring Human Resources Management student se...,"Houston, Texas Area",500+
24,25,Student at Humber College and Aspiring Human R...,Kanada,61
36,37,Student at Humber College and Aspiring Human R...,Kanada,61
68,69,"Director of Human Resources North America, Gro...","Greater Grand Rapids, Michigan Area",500+
49,50,Student at Humber College and Aspiring Human R...,Kanada,61
51,52,Student at Humber College and Aspiring Human R...,Kanada,61
78,79,Liberal Arts Major. Aspiring Human Resources A...,"Baton Rouge, Louisiana Area",7
26,27,Aspiring Human Resources Management student se...,"Houston, Texas Area",500+
72,73,"Aspiring Human Resources Manager, seeking inte...","Houston, Texas Area",7


In [135]:
## XGBoost model

XGB_ranker = xgb.XGBRanker(
    objective='rank:ndcg',
    random_state=SEED
    )

XGB_ranker.fit(
    X_train,
    y_train,
    group=train_group
    )

XGBRanker(base_score=None, booster=None, callbacks=None, colsample_bylevel=None,
          colsample_bynode=None, colsample_bytree=None, device=None,
          early_stopping_rounds=None, enable_categorical=False,
          eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
          importance_type=None, interaction_constraints=None,
          learning_rate=None, max_bin=None, max_cat_threshold=None,
          max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
          max_leaves=None, min_child_weight=None, missing=nan,
          monotone_constraints=None, multi_strategy=None, n_estimators=None,
          n_jobs=None, num_parallel_tree=None, random_state=7247, ...)

In [136]:
xgb_preds = XGB_ranker.predict(sorted_final_df)

XGB_ranker_predictions = pd.DataFrame({'Candidate_ID': sorted_final_df_id, 'Predictions': xgb_preds})
XGB_ranker_predictions = XGB_ranker_predictions.sort_values(by='Predictions', ascending=False)

preds = create_multilabels(xgb_preds)
accuracy_score = acc(sorted_final_df_label, preds)
print(f"Accuracy score: {accuracy_score}")

og_df.loc[XGB_ranker_predictions['Candidate_ID']-1].head(40)

Accuracy score: 0.7307692307692307


,id,job_title,location,connection
5,6,Aspiring Human Resources Specialist,Greater New York City Area,1
32,33,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44
23,24,Aspiring Human Resources Specialist,Greater New York City Area,1
65,66,Experienced Retail Manager and aspiring Human ...,"Austin, Texas Area",57
72,73,"Aspiring Human Resources Manager, seeking inte...","Houston, Texas Area",7
26,27,Aspiring Human Resources Management student se...,"Houston, Texas Area",500+
51,52,Student at Humber College and Aspiring Human R...,Kanada,61
49,50,Student at Humber College and Aspiring Human R...,Kanada,61
38,39,Student at Humber College and Aspiring Human R...,Kanada,61
36,37,Student at Humber College and Aspiring Human R...,Kanada,61


In [137]:
# XGB_ranker.save_model('../Save_Models/xgb_ranker.json')

In [138]:
## mean method
XGB_ranker_2 = xgb.XGBRanker(
    lambdarank_pair_method='mean',
    lambdarank_num_pair_per_sample=16,
    objective='rank:ndcg',
    random_state=SEED,
    lambdarank_normalization=False,
    eval_metric='ndcg-',
    learning_rate=.01
)

XGB_ranker_2.fit(
    X_train,
    y_train,
    group=train_group
)

XGBRanker(base_score=None, booster=None, callbacks=None, colsample_bylevel=None,
          colsample_bynode=None, colsample_bytree=None, device=None,
          early_stopping_rounds=None, enable_categorical=False,
          eval_metric='ndcg-', feature_types=None, gamma=None, grow_policy=None,
          importance_type=None, interaction_constraints=None,
          lambdarank_normalization=False, lambdarank_num_pair_per_sample=16,
          lambdarank_pair_method='mean', learning_rate=0.01, max_bin=None,
          max_cat_threshold=None, max_cat_to_onehot=None, max_delta_step=None,
          max_depth=None, max_leaves=None, min_child_weight=None, missing=nan,
          monotone_constraints=None, multi_strategy=None, n_estimators=None, ...)

In [139]:
xgb2_preds = XGB_ranker_2.predict(sorted_final_df)

XGB_ranker_2_predictions = pd.DataFrame({'Candidate_ID': sorted_final_df_id, 'Predictions': xgb2_preds})
XGB_ranker_2_predictions = XGB_ranker_2_predictions.sort_values(by='Predictions', ascending=False)

preds = create_multilabels(xgb2_preds)
accuracy_score = acc(sorted_final_df_label, preds)
print(f"Accuracy score: {accuracy_score}")

og_df.loc[XGB_ranker_2_predictions['Candidate_ID']-1].head(40)

Accuracy score: 0.7115384615384616


,id,job_title,location,connection
5,6,Aspiring Human Resources Specialist,Greater New York City Area,1
71,72,Business Management Major and Aspiring Human R...,"Monroe, Louisiana Area",5
23,24,Aspiring Human Resources Specialist,Greater New York City Area,1
51,52,Student at Humber College and Aspiring Human R...,Kanada,61
49,50,Student at Humber College and Aspiring Human R...,Kanada,61
38,39,Student at Humber College and Aspiring Human R...,Kanada,61
36,37,Student at Humber College and Aspiring Human R...,Kanada,61
24,25,Student at Humber College and Aspiring Human R...,Kanada,61
8,9,Student at Humber College and Aspiring Human R...,Kanada,61
6,7,Student at Humber College and Aspiring Human R...,Kanada,61


In [140]:
## topk method
XGB_ranker_3 = xgb.XGBRanker(
    lambdarank_pair_method='topk',
    lambdarank_num_pair_per_sample=10,
    objective='rank:ndcg',
    random_state=SEED,
    lambdarank_normalization=False,
    eval_metric='ndcg-',
    learning_rate=.01
)

XGB_ranker_3.fit(
    X_train,
    y_train,
    group=train_group
)

XGBRanker(base_score=None, booster=None, callbacks=None, colsample_bylevel=None,
          colsample_bynode=None, colsample_bytree=None, device=None,
          early_stopping_rounds=None, enable_categorical=False,
          eval_metric='ndcg-', feature_types=None, gamma=None, grow_policy=None,
          importance_type=None, interaction_constraints=None,
          lambdarank_normalization=False, lambdarank_num_pair_per_sample=10,
          lambdarank_pair_method='topk', learning_rate=0.01, max_bin=None,
          max_cat_threshold=None, max_cat_to_onehot=None, max_delta_step=None,
          max_depth=None, max_leaves=None, min_child_weight=None, missing=nan,
          monotone_constraints=None, multi_strategy=None, n_estimators=None, ...)

In [141]:
xgb3_preds = XGB_ranker_3.predict(sorted_final_df)

XGB_ranker_3_predictions = pd.DataFrame({'Candidate_ID': sorted_final_df_id, 'Predictions': xgb3_preds})
XGB_ranker_3_predictions = XGB_ranker_3_predictions.sort_values(by='Predictions', ascending=False)

preds = create_multilabels(xgb3_preds)
accuracy_score = acc(sorted_final_df_label, preds)
print(f"Accuracy score: {accuracy_score}")

og_df.loc[XGB_ranker_3_predictions['Candidate_ID']-1].head(40)

Accuracy score: 0.7019230769230769


,id,job_title,location,connection
5,6,Aspiring Human Resources Specialist,Greater New York City Area,1
16,17,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44
51,52,Student at Humber College and Aspiring Human R...,Kanada,61
49,50,Student at Humber College and Aspiring Human R...,Kanada,61
38,39,Student at Humber College and Aspiring Human R...,Kanada,61
36,37,Student at Humber College and Aspiring Human R...,Kanada,61
24,25,Student at Humber College and Aspiring Human R...,Kanada,61
8,9,Student at Humber College and Aspiring Human R...,Kanada,61
6,7,Student at Humber College and Aspiring Human R...,Kanada,61
57,58,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44


In [142]:
# XGB_ranker = xgb.XGBRanker()
# XGB_ranker_model = XGB_ranker.load_model('../Save_Models/xgb_ranker.json')
# type(XGB_ranker_model)

In [143]:
## this function will recursively rank a pre-sorted pd.Dataframe 
## this function assumes that the first dataframe is a vectorized representation of the candidates' job titles along with other features that will be used for prediction
## the second dataframe will be the original dataset that will be used for the final ranking of candidates

def get_candidates(sorted_df, dataset, model, num_cands=10, index_list=[]):
    '''
    --Parameters--
    sorted_df: pd.Dataframe that has been pre-ranked for the job title search term
    dataset: pd.Dataframe that is the original un-ranked dataset of the candidates
    num_cands: number of times the ranking process will be applied to the pre-ranked dataset - default=10

    --Return--
    dataset that is indexed by the top candidates from each round of rankings
    '''

    indices = index_list

    predictions = model.predict(sorted_df)
    predictions = pd.DataFrame({'candidate_index': sorted_df.index, 'predictions': predictions})
    sorted_predictions = predictions.sort_values(by='predictions', ascending=False)

    top_candidate_index = sorted_predictions['candidate_index'].values[0]
    indices.append(top_candidate_index)
    # print('Top candidate:', top_candidate_index)

    rerank_indeces = sorted_predictions['candidate_index'][1:].values
    sorted_df = sorted_df.loc[rerank_indeces]

    if num_cands == 0:
        print(dataset.loc[indices])
    else:
        get_candidates(sorted_df=sorted_df, dataset=og_df, model= model, num_cands=num_cands-1, index_list=indices)

In [144]:
top_candidates = get_candidates(sorted_df=sorted_final_df, dataset=og_df, model=XGB_ranker, num_cands=10)
top_candidates

    id                                          job_title  \
5    6                Aspiring Human Resources Specialist   
32  33              Aspiring Human Resources Professional   
20  21              Aspiring Human Resources Professional   
49  50  Student at Humber College and Aspiring Human R...   
28  29  Aspiring Human Resources Management student se...   
48  49                Aspiring Human Resources Specialist   
16  17              Aspiring Human Resources Professional   
13  14  2019 C.T. Bauer College of Business Graduate (...   
0    1  2019 C.T. Bauer College of Business Graduate (...   
43  44  2019 C.T. Bauer College of Business Graduate (...   
26  27  Aspiring Human Resources Management student se...   

                               location connection  
5            Greater New York City Area          1  
32  Raleigh-Durham, North Carolina Area         44  
20  Raleigh-Durham, North Carolina Area         44  
49                               Kanada         61  
28

## Prompt Engineering

Using some of the latest large laguage models (LLM)'s, we will 'prompt' these models to see if we can obtain results like the ones above with only a few lines of code.

The down side of this method is that the generating time is over 2 hours. With better hardware (GPU's) the model might not take as long.

For more information on fine-tuning the Phi-3 model you can follow this link: https://huggingface.co/microsoft/Phi-3-mini-4k-instruct

In [99]:
candidates = og_df['job_title'].tolist()
candidates_ids = og_df['id'].tolist()

candidates_list = list(zip(candidates_ids, candidates)) ## list of tuples that correspond to the candidate id and their job title

In [100]:
import transformers
from transformers.generation import CompileConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import flash_attention

model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-4k-instruct", torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

messages = [
    {
        "role": "system",
        "content": "You are a ranking algorithm that ranks a list of items based on how similar each item is to a given phrase and returns the top 'n' items that the user requests."},
    {
        "role": "user", 
        "content": f"In this list of items {candidates_list}, give me the top 15 items that are related to the phrase 'aspiring human resources'."}
]

model_inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
# print(model_inputs.shape)
input_length = model_inputs.shape[1]
generated_ids = model.generate(model_inputs, do_sample=True, max_new_tokens=650)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Python(14281) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  6.14it/s]
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48
You are not running the flash-attention implementation, expect numerical differences.


In [101]:
print(tokenizer.batch_decode(generated_ids[:, input_length:], skip_special_tokens=True)[0])

Given the instruction to rank items in relation to the keyword 'aspiring human resources' from an initially sorted list (items 1-104), let's extract and rank them to find the top 15 most related to that theme.

Here are the top 15 items related to 'aspiring human resources':

1. Aspiring Human Resources Professional
2. HR Senior Specialist at Ryan
3. Aspiring Human Resources Professional
4. Senior HR Business Partner at EY
5. Aspiring Human Resources Management Major
6. HR Generalist at Schwan's
7. Aspiring Human Resources Professional | Passionate about helping to create an inclusive and engaging work environment
8. HR Manager at Endemol Shine North America
9. HR Professional for GIS software industry
10. HR Specialist at Heil Environmental
11. Aspiring Human Resources Analyst
12. Aspiring Human Resources Manager
13. Student at Humber College and Aspiring Human Resources Generalist
14. Seeking Human Resources Position in St. Louis
15. Aspiring Human Resources Professional – Seeking En

In [102]:
model.save_pretrained('../Save_Models/phi3_tuned', from_pt=True)
tokenizer.save_pretrained('../Save_Models/phi3_tuned')

Python(16181) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[2025-01-16 13:18:01,004] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to mps (auto detect)


W0116 13:18:01.392534 2136 site-packages/torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


('../Save_Models/phi3_tuned/tokenizer_config.json',
 '../Save_Models/phi3_tuned/special_tokens_map.json',
 '../Save_Models/phi3_tuned/tokenizer.json')

In [ ]:
## for loading back in the saved models

model = AutoModelForCausalLM.from_pretrained("../Save_Models/phi3_tuned", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained("../Save_Models/phi3_tuned")

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.40it/s]
